In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
spark_context = SparkContext("local", "first_app")
spark = SparkSession(spark_context)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sql_context = SQLContext(spark_context)

In [2]:
text_file = sql_context.read.format("com.databricks.spark.csv").options(header='true', inferschema='true', quote='"', delimiter=',').load("/home/jovyan/work/TraficoAereoGt.csv")
rdd_filter = text_file.rdd.map(tuple)
rdd_landings = rdd_filter.map(lambda word: (word[2], word[3]))
rdd_landings.take(5)

[('Coatepeque', 3607),
 ('Coatepeque', 5261),
 ('Coatepeque', 629),
 ('Coatepeque', 3690),
 ('Coatepeque', 307)]

In [3]:
rdd_landings = rdd_landings.reduceByKey(lambda a,b: a+b)
print(rdd_landings.collect())

[('Coatepeque', 167223), ('Huehuetenango', 198734), ('Poptun', 173559), ('Puerto Barros', 203204), ('Mundo Maya', 170385), ('La Aurora', 150513), ('Puerto San Jose', 162211), ('Quetzaltenango', 194403), ('Retalhuleu', 174374), ('San Marcos', 208806)]


In [4]:
rdd_landings = spark_context.parallelize(rdd_landings.sortBy(lambda a: a[1], False).collect())
print(rdd_landings.collect())

[('San Marcos', 208806), ('Puerto Barros', 203204), ('Huehuetenango', 198734), ('Quetzaltenango', 194403), ('Retalhuleu', 174374), ('Poptun', 173559), ('Mundo Maya', 170385), ('Coatepeque', 167223), ('Puerto San Jose', 162211), ('La Aurora', 150513)]


In [5]:
airports_name = rdd_landings.map(lambda x: x[0])
airports_landings = rdd_landings.map(lambda x: x[1])
print(airports_name.collect())
print(airports_landings.collect())

['San Marcos', 'Puerto Barros', 'Huehuetenango', 'Quetzaltenango', 'Retalhuleu', 'Poptun', 'Mundo Maya', 'Coatepeque', 'Puerto San Jose', 'La Aurora']
[208806, 203204, 198734, 194403, 174374, 173559, 170385, 167223, 162211, 150513]


In [6]:
fig = go.Figure(data=go.Bar(x=airports_name.collect(),y=airports_landings.collect()))
fig.update_layout(title_text='Total landings per airport',title_font_size=30,
                  yaxis=dict(title='Landings',title_font_size=25),
                  xaxis=dict(title='Airport',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('report3.html', auto_open=True)